In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from datetime import datetime

import numpy as np
from collections import Counter
import os
from argparse import Namespace
import random
import json

flags = Namespace(
    train_file='blacckards.json',
    seq_size=9,
    batch_size=128,
    embedding_size=100,
    lstm_size=6,
    gradients_norm=5,
    predict_top_k=3,
)

In [2]:
##Loading all black cards into our program
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as f:
        cah_data = json.load(f)
    corpus = ''
    for card in cah_data['blackCards']:
        corpus += card['text'] + ' '
    corpus.replace('.', ' . ')
    corpus.replace(',', ' , ')
    corpus.replace('?', ' ? ')
    corpus = corpus.split()
    
    #print(corpus)
    first_words = ''
    for card in cah_data['blackCards']:
        #print(card['text'].split()[0])
        first_words += (card['text'].split())[0] + ' '
    first_words.split()
    
    word_counts = Counter(corpus)

    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    int_text = [vocab_to_int[w] for w in corpus]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text #,words_to_use_as_init

In [3]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [4]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [5]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [6]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(torch.int64).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])
    
    words_new = []
    for initwords in words:
        words_new.append(initwords)
        
    words_new.append(int_to_vocab[choice])
    for _ in range(8):
        ix = torch.tensor([[choice]]).to(torch.int64).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words_new.append(int_to_vocab[choice])
    print(' '.join(words_new))

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags.train_file, flags.batch_size, flags.seq_size)

net = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.001)

##I chose the first few first words from the original set; some of them seem to work well.
words_to_use_as_init = ['_', 'I', 'What', 'This', 'TSA', 'Alternative', "It's", "What's", 'In', '_.', 'Next',
                        'Instead', 'A', 'White',  'During',  'When', 'While', 'But', 'Coming']
iteration = 0
epoch_count=100000
max_accuracy = 50
print("I actually started the cycle.")
for e in range(epoch_count):
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = net.zero_state(flags.batch_size)

    # Transfer data to GPU
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1
        net.train()
        optimizer.zero_grad()

        # Convert data to correct dtype, not using GPU anyway, so can't really send to it.
        x = torch.tensor(x).to(torch.int64).to(device)
        y = torch.tensor(y).to(torch.int64).to(device)

        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()
        accuracy =(1-loss_value)*100
        # Perform back-propagation
        loss.backward()
        
        _ = torch.nn.utils.clip_grad_norm_(
            net.parameters(), flags.gradients_norm)

        # Update the network's parameters
        optimizer.step()
        if iteration % 500 == 0:
            now = datetime.now()
            initial_words = []
            for iwords_count in range(1):
                initial_words.append(random.choice(words_to_use_as_init))

            predict(device, net, initial_words, n_vocab,
                            vocab_to_int, int_to_vocab, top_k=3)
            current_time = now.strftime("%H:%M:%S")
            print(current_time, ' -Epoch: {}/{}'.format(e, epoch_count), 'Accuracy: {}%'.format(accuracy))
    if accuracy > max_accuracy:
        break

creations = 0
while creations < 30:
    initial_words = []
    for iwords_count in range(1):
        initial_words.append(random.choice(words_to_use_as_init))

    predict(device, net, initial_words, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=3)
    creations += 1


I actually started the cycle.
_. I you for my Barack for in _. you
14:17:49  -Epoch: 124/100000 Accuracy: -483.45179557800293%
When _ the to in a in with the _.
14:17:55  -Epoch: 249/100000 Accuracy: -416.3238525390625%
What's _. I'm sorry, your Beginning this When is a
14:18:02  -Epoch: 374/100000 Accuracy: -380.17144203186035%
_ and the to this out can by the you
14:18:09  -Epoch: 499/100000 Accuracy: -354.4942855834961%
What has of _. is _. is _. is the
14:18:16  -Epoch: 624/100000 Accuracy: -334.3335151672363%
In with of _. is a in with the _.
14:18:22  -Epoch: 749/100000 Accuracy: -318.78390312194824%
Instead with of to the new show, date, my Really
14:18:29  -Epoch: 874/100000 Accuracy: -305.086088180542%
_. I for _. I'm and and _. If What
14:18:35  -Epoch: 999/100000 Accuracy: -293.72591972351074%
It's and and a in my me that and _.
14:18:41  -Epoch: 1124/100000 Accuracy: -283.724045753479%
While _. What will you _ in my Barack plays
14:18:48  -Epoch: 1249/100000 Accuracy: -275.

Coming to your burps door using and back be out
14:25:56  -Epoch: 9624/100000 Accuracy: -128.52725982666016%
Instead going the way your burps vision, love it do
14:26:02  -Epoch: 9749/100000 Accuracy: -128.87465953826904%
_. If is we are needed $1,000. days days how
14:26:08  -Epoch: 9874/100000 Accuracy: -127.52254009246826%
A study must bad sex. I you'd about a man
14:26:15  -Epoch: 9999/100000 Accuracy: -126.7514705657959%
What keeps me _. In _. In _ is _.
14:26:21  -Epoch: 10124/100000 Accuracy: -126.61252021789551%
White _." Why am if I I'm me. Hi, the
14:26:27  -Epoch: 10249/100000 Accuracy: -125.32196044921875%
In that In being their with out friend with over
14:26:34  -Epoch: 10374/100000 Accuracy: -129.729962348938%
When I'm not only me a powerful because found is
14:26:40  -Epoch: 10499/100000 Accuracy: -125.46777725219727%
During devoted at own an night? doses. role, Daniel Day-Lewis
14:26:46  -Epoch: 10624/100000 Accuracy: -124.84164237976074%
What will not be is _. Who Goo

White whole final wanted. White States, am I saw now
14:33:52  -Epoch: 18874/100000 Accuracy: -96.37036323547363%
Next date, Major discover a _!" without eat _. In
14:33:58  -Epoch: 18999/100000 Accuracy: -92.20346212387085%
White people supercharged it _? Puberty you must overcome have
14:34:06  -Epoch: 19124/100000 Accuracy: -92.88632869720459%
During Calendar alert: I on your Kung energy? some League
14:34:13  -Epoch: 19249/100000 Accuracy: -93.25964450836182%
During America. What would will by and _ to _.
14:34:20  -Epoch: 19374/100000 Accuracy: -91.50131940841675%
A romantic, produced stars Sean Penn ready of coal, Santa
14:34:27  -Epoch: 19499/100000 Accuracy: -91.37221574783325%
Next disturbing, buffalo statue to this morning, _? relationship? MTV's
14:34:34  -Epoch: 19624/100000 Accuracy: -91.62695407867432%
But if am we don't do want for the world
14:34:41  -Epoch: 19749/100000 Accuracy: -90.20894765853882%
What keeps me on the mood don't did you on
14:34:48  -Epoch: 19874/100

This join because but I Young of for with crying
14:42:07  -Epoch: 27874/100000 Accuracy: -74.32267665863037%
While porn star name they very one become Mommy me
14:42:14  -Epoch: 27999/100000 Accuracy: -74.70844984054565%
But thought we no What <i>you</i> requires _. The _
14:42:21  -Epoch: 28124/100000 Accuracy: -76.10268592834473%
Instead of people his said, next level? down _, but
14:42:28  -Epoch: 28249/100000 Accuracy: -71.06797695159912%
White whole panel Roosevelt must his You with over have
14:42:35  -Epoch: 28374/100000 Accuracy: -72.38513231277466%
TSA Ray's hit You answering? The Five using _! Now
14:42:41  -Epoch: 28499/100000 Accuracy: -69.98928785324097%
What's say like _. What am _ in talk to
14:42:48  -Epoch: 28624/100000 Accuracy: -70.93580961227417%
But thought find me, but _ here got a ton
14:42:55  -Epoch: 28749/100000 Accuracy: -77.10500955581665%
_ I will finally doses. for a 50-foot Hi tribe
14:43:02  -Epoch: 28874/100000 Accuracy: -71.6984510421753%
Next disturb

White _." The What you has and for _. This
14:50:17  -Epoch: 36874/100000 Accuracy: -58.41001272201538%
_ have _. I'm sorry, Professor, gas? I he found
14:50:24  -Epoch: 36999/100000 Accuracy: -56.55837059020996%
This class more? ready am it's one at most extreme
14:50:31  -Epoch: 37124/100000 Accuracy: -57.343268394470215%
Coming _ in _, solace out my asshole Watch you
14:50:38  -Epoch: 37249/100000 Accuracy: -60.85995435714722%
Instead of this back? months clear of _. It's but
14:50:44  -Epoch: 37374/100000 Accuracy: -56.872761249542236%
When along I've I on the the last by and
14:50:51  -Epoch: 37499/100000 Accuracy: -60.70575714111328%
Alternative Achievement Friday not delivery. It's and Our fat as
14:50:58  -Epoch: 37624/100000 Accuracy: -54.647135734558105%
Instead going _. Coming to a problems with _! but
14:51:05  -Epoch: 37749/100000 Accuracy: -57.39196538925171%
During now, fend off ninjas Christmas, an offering of people
14:51:12  -Epoch: 37874/100000 Accuracy: -56.67277574

...

TSA guidelines _. Betcha eyes. better discovers ask? _? gives
15:15:43  -Epoch: 64124/100000 Accuracy: -31.487679481506348%
But if you left what you with out friend buffalo
15:15:50  -Epoch: 64249/100000 Accuracy: -27.475297451019287%
_. Life parent's so I _ ruin hurt, Netflix: lover,
15:15:57  -Epoch: 64374/100000 Accuracy: -30.737435817718506%
In the trauma. Lunge, news FOR came sex. prepare for
15:16:04  -Epoch: 64499/100000 Accuracy: -36.68848276138306%
In the trauma. final plan face What out of _.
15:16:11  -Epoch: 64624/100000 Accuracy: -33.048152923583984%
Alternative A Money parks in goods and Siskel a 50-foot
15:16:18  -Epoch: 64749/100000 Accuracy: -27.133619785308838%
_ have a non-stop halt? is eyes catch chocolate, three
15:16:24  -Epoch: 64874/100000 Accuracy: -28.28958034515381%
But It beer, is enjoy what is a making new
15:16:31  -Epoch: 64999/100000 Accuracy: -34.72280502319336%
White without then about a of for in the workplace.
15:16:39  -Epoch: 65124/100000 Accuracy:

But no me doctor, is the the is a billionaire,
15:23:50  -Epoch: 73124/100000 Accuracy: -24.48326349258423%
_ and now picketing soldiers' Practice." correlation between But if
15:23:57  -Epoch: 73249/100000 Accuracy: -22.073698043823242%
A study Jail, life is about a man love my
15:24:04  -Epoch: 73374/100000 Accuracy: -20.622718334197998%
When I world said, things has only / season, be
15:24:11  -Epoch: 73499/100000 Accuracy: -20.0303316116333%
It's enough by the "Everything on Madam, Morphin' startup. rich
15:24:17  -Epoch: 73624/100000 Accuracy: -19.655656814575195%
While about _. During show _! if I who's for
15:24:23  -Epoch: 73749/100000 Accuracy: -19.82203722000122%
While HBO's new show, Cooking with the bit of crying
15:24:30  -Epoch: 73874/100000 Accuracy: -19.18727159500122%
Instead Jordan to his I'm _. This out Rumor of
15:24:36  -Epoch: 73999/100000 Accuracy: -19.309687614440918%
Coming for the poolside. "This is madness." immersed request _
15:24:42  -Epoch: 74124/100000 A

Next complete the We're Bobby grateful! social Willis book. The
15:31:18  -Epoch: 82124/100000 Accuracy: -12.783205509185791%
During The join hands Americans _ will you _! What
15:31:24  -Epoch: 82249/100000 Accuracy: -13.773167133331299%
White without _. Tonight a powerful conceived" Tuesday, remember: a
15:31:30  -Epoch: 82374/100000 Accuracy: -22.68928289413452%
While we he sense that? What's heaven? _. It's full
15:31:37  -Epoch: 82499/100000 Accuracy: -14.262807369232178%
TSA one. But wait, there's more! What have fuck ruled
15:31:43  -Epoch: 82624/100000 Accuracy: -18.047702312469482%
A wise Armani Third news organizations. Because they trade legs
15:31:49  -Epoch: 82749/100000 Accuracy: -12.43375539779663%
It's trade 200 cigarettes to this When you am the
15:31:55  -Epoch: 82874/100000 Accuracy: -13.791930675506592%
_ have but the the new DLC for Mass boy.
15:32:01  -Epoch: 82999/100000 Accuracy: -12.357616424560547%
_. Life of Hey baby, don't mean to hurt fix
15:32:07  -Epoch: 

What keeps me warm with _. I've _. Next our
15:38:50  -Epoch: 91249/100000 Accuracy: -8.792245388031006%
But if is think he if is America's famous spas,
15:38:56  -Epoch: 91374/100000 Accuracy: -12.077581882476807%
What's Wall Today national other with of a vote to
15:39:02  -Epoch: 91499/100000 Accuracy: -8.580505847930908%
I have tonight! You can on live discovered that by
15:39:09  -Epoch: 91624/100000 Accuracy: -7.558763027191162%
While those buffalo wings situation Carrey road. little-known own we
15:39:15  -Epoch: 91749/100000 Accuracy: -14.063191413879395%
A wise ready Armani film evil; like _! But before
15:39:21  -Epoch: 91874/100000 Accuracy: -7.072973251342773%
What's famously discover your eyes child next dead-on tourism colony?
15:39:27  -Epoch: 91999/100000 Accuracy: -7.6021552085876465%
Alternative relationship year's Omegathon, Night "_". favorite porn God, under
15:39:33  -Epoch: 92124/100000 Accuracy: -3.906857967376709%
White making life evidence Batman's fails, My w

While about _, happened teacher will never on friends, _.
15:46:02  -Epoch: 99999/100000 Accuracy: -2.2469401359558105%
This Don't Children, we program, way involved over? nor Street
This Wake bargaining, 1,000 favorite delicacy I In to Unfortunately,
This Don't repressed Tim Cream ask? ladies on World Series
What is. You that makes during or CNN hurt, ain't
But no _ goes will never be next level? and
I have about _. Why can't Man, for a billionaire,
But no problems but masturbate like and back got I
What is. You or Ladies drink <i>Cooking unfair Practice." face
This Don't repressed energy into too found you in _.
Coming it with _. when never and the United States,
What's doesn't parks for _! _ and _ _ _
When Why and It's but what not will turn can
White making life evidence Batman's cheer try struggle from fails,
I I for with _. Having trap! As king, how
It's on Mrs. Mastadon! Cream home sweet everything are Sex
Instead club. Money We been How succeed, normal. we do
Next buffalo wings

In [11]:
words_to_use_as_init = ['_', 'I', 'What', 'This', 'TSA', 'Alternative', "It's", "What's", 'In', '_.', 'Next',
                        'Instead', 'A', 'White',  'During',  'When', 'While', 'But', 'Coming']
creations = 0
while creations < 500:
    initial_words = []
    for iwords_count in range(1):
        initial_words.append(random.choice(words_to_use_as_init))

    predict(device, net, initial_words, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=3)
    creations += 1

TSA sky. What come killed all eternity. As king, how
A Story living of a commence endure involvement of a
What's doesn't son Obama door. handle that you was _
It's on sir, called me? now died, are on Netflix
Instead has for changing me Betrayal. your captain debate, the
Next an collisions between paper IV can Easy _! but
This I'm Try and I In to Clinton's time? _
Alternative grateful! relationship offer _! It beer, _ Ebert was
It's Morphin' minutes. they Now, can by on _starter. and
This I'm looked _ to talk defeated me? Well, penis?
Alternative relationship When pull As reparations of Lent? fuck to
It's enough by _: I last all to _. Life's
While those buffalo stations! unmoved, complete America. years. You on
A &rarr; situation rats suit: raps Church child can't tells
Alternative relationship summer, freaked is America's club and _. Blessed
White was her an Dreams an outlaw _. I learned
Alternative grateful! good, you It into we don't Do up
What's doesn't son Obama door. To prepare It

White was take day Action Action stations! Set stations! Set
Alternative grateful! While can't gives ain't What's the United States,
During _." In sauna? hand _, you don't pay for
This I'm Try into black, Nothing some having condition What's
A &rarr; $300. After Jackson's prohibit next 15 Unfortunately, idea
_. Life parent's immersed Cancel swings my WHOOO! gave this
_. Coming for _. The the way out been _.
Instead of his midlife the sex, me we I only
Coming it from close prove at hungers for _. And
What's doesn't eyes. That BBQ regimen, Anderson Let's meetings. Bond,
When Why _ BOT<br><br>EXECUTING God. look for going _. Could
But It see ends officially always along _." now truly
During _." In his childhood, address, George lack film will
I to Christmas. relationship so grateful! While all infinity, plan
Instead club. 2 Adventure.<br>Romance.<br>_.<br><br>From Paramount Because III on see can't
But no _ again. Today on the cold, Dear vision,
I to few times. scientists believe Man feet

A Story living ruined The powerful looked fuck a princess
I I been slippery explore ends going _. I'm stay
A wise man reparations ready ready of pressure, the seventh
In his introduced know vote? name Lunge, because a ton
What keeps and _: did you _: Good It's _.
But if you want can't handle _. How did Stella
When end This holiday season, hot, and What's the mood
A wise man B it's be involved evidence evidence with
What is. You that lab kids / season, be enough
_ that believe who can't stand that would _ be
TSA Wake up, vote? White fun. room. become PAX take
This I'm Try and I There is to outlaw the
I have a for has been guilty been living my
During Americans In the third the bearded anger, Here have
Coming to about my WHOOO! gave this Don't repressed complicate
I to few child about my pussy. But before I
_ have a non-stop Ray's five, bro. During Well, Astronomers
It's enough by _: know guys, successful After "Mom's to-do
Coming it with the third take way about _ I
When I This White It

This Don't exactly students his introduced want Five Stages for
In was over, ruined _ have _. As reparations Mountain
White 'tis asleep to dark Oprah's book for was all
But It lurks truly non-stop infinite wisdom an even like
_. Life hurt totally Tiger! with show to the city
In the cards been one time right The What new
Alternative good, It lurks truly non-stop infinite Sir, those during
Next buffalo stuff _." _. Google embracing music, cold, Dear
A wise ready Armani Your trouble anyone. tired Batman's months
_ I to Christmas. relationship A album, breakthrough, scientists I
It's enough I me _, I'll from called Beginning surprise
Next an exhibit on _. subjecting could _ forever. How
Coming to take _. _: A wise rings. only are
Instead club. Money on forget! World War III will and
Next buffalo unmoved, Baseball "Barack _!" Movie, that? secret infinite
This Don't repressed Tim Sabertooth princess kissed most liked _?
_ I got for the next level? the have been
I have about my My You guys, k

Almost good:
During sex, I got rhythm, I've got _. The blind
I was a cantidate who is openly _.


No blanks:
While we find this tiny penis? this virginity? Tinder, but
This season on Man vs. Wild, bear contract, Obama.

Straight out weird:
_. Tonight we _. An international attack Something hurt
_ in country, _, and my butt?
Instead it hungers for flesh.
